In [12]:
!pip install geocoder

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 98.6/98.6 kB 939.9 kB/s eta 0:00:00
     -------------------------------------- 829.2/829.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for future, since package 'wheel' is not installed.
  Running setup.py install for future: started
  Running setup.py install for future: finished with status 'done'


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import geopy.geocoders
from geopy.geocoders import Nominatim 

def address_to_coordinates(address):
    geolocator = geopy.geocoders.Nominatim(user_agent='geoapiExercises')
    location = geolocator.geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

coordinates = address_to_coordinates('台北市信義區逸仙路50巷22號')
print(coordinates)  # (37.4224764, -122.0842499)


None


In [15]:
import geocoder
g = geocoder.google('台北市信義區逸仙路50巷22號')
print(g.latlng)

None


In [16]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Mountain View, CA'}
r = requests.get(url, params=params)
results = r.json()['results']
location = results[0]['geometry']['location']
location['lat'], location['lng']

IndexError: list index out of range

In [4]:
import datetime
# 取得當前日期
today = datetime.datetime.today().date()
print(today)

# 計算隔天日期
tomorrow = today + datetime.timedelta(days=1)

# 輸出隔天日期
print(tomorrow)

2023-01-03
2023-01-04


In [9]:
!pip3 install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 242.1/242.1 kB 1.6 MB/s eta 0:00:00


In [38]:
import pandas as pd
import openpyxl
def population(loc):
    df = pd.read_excel('C:/Users/student/Desktop/my_project/111年09月 ( 鄉鎮市區 )人口密度.xlsx')
    x = df.loc[df['地區']==f'{loc}',['人口密度'][0]]
    return x

In [39]:
population('新北市永和區')

0    36885.26
Name: 人口密度, dtype: float64